The Wedin bound and random direction samples are embarassingly parallel and can be the main computational bottleneck in AJIVE. These can be done in parallel in two different ways: using multiple cores on one computer or over multiple computers. The AJIVE object easily handles the former of these using `sklearn.externals.joblib.Parallel`. For the latter, the user can precompute the wedin and random direction samples then input these to the AJIVE object.

This notebook gives examples of both.

In [1]:
import numpy as np

from jive.ajive_fig2 import generate_data_ajive_fig2
from jive.AJIVE import AJIVE
from jive.PCA import PCA

In [2]:
X, Y = generate_data_ajive_fig2()

# Parallelizing over cores in a computer (esay!)

In [3]:
# no parallelization
%time AJIVE(init_signal_ranks=[2, 3]).fit(blocks=[X, Y])

# using parallelization, simply set n_jobs parameter
%time AJIVE(init_signal_ranks=[2, 3], n_jobs=-1).fit(blocks=[X, Y])

CPU times: user 13.5 s, sys: 1.85 s, total: 15.3 s
Wall time: 4.25 s
CPU times: user 793 ms, sys: 243 ms, total: 1.04 s
Wall time: 4.38 s


AJIVE, joint rank: 1, block 0 indiv rank: 1, block 1 indiv rank: 2

# Parallelizing over computers (still pretty easy)

#### pre-computing the initial_svd

This is not scrictly necessary for parallel computation, but will make AJIVE faster.

In [4]:
x_pca = PCA(n_components=5).fit(X)
y_pca = PCA(n_components=10).fit(Y)

# pca.get_UDV() returns scores, singular values, loadings i.e. a three tuple
print(len(x_pca.get_UDV()))

AJIVE(init_signal_ranks=[2, 3]).fit(blocks=[X, Y],
                                    precomp_init_svd=[x_pca.get_UDV(), y_pca.get_UDV()])

3


AJIVE, joint rank: 1, block 0 indiv rank: 1, block 1 indiv rank: 2

#### sample wedin and random directions

In [5]:
from jive.wedin_bound import get_wedin_samples
from jive.random_direction import sample_randdir

In [6]:
def one_computer_wedin_samples(blocks, precomp_init_svd, init_signal_ranks, R):
    wedin_samples = {}
    for bn in blocks.keys():
        wedin_samples[bn] = get_wedin_samples(X=blocks[bn],
                                              U=precomp_init_svd[bn][0], # scores
                                              D=precomp_init_svd[bn][1], # svals
                                              V=precomp_init_svd[bn][2], # loadings
                                              rank=init_signal_ranks[bn], 
                                              R=R)
    
    return wedin_samples

In [7]:
samples_per_computer = 10
NUM_COMPS = 100
blocks = {'x': X, 'y': Y}
init_signal_ranks = {'x': 2, 'y': 3}

num_obs = blocks['x'].shape[0]
signal_ranks=list(init_signal_ranks.values())

x_pca = PCA().fit(blocks['x'])
y_pca = PCA().fit(blocks['y'])
precomp_init_svd = {'x': x_pca.get_UDV(), 'y': y_pca.get_UDV()}

wedin_samples = {bn: np.array([]) for bn in blocks.keys()}
random_samples = np.array([])

for _ in range(NUM_COMPS): # pretend each iteration is a different computer
    comp_rand = sample_randdir(num_obs, signal_ranks, R=samples_per_computer)
    comp_wedin = one_computer_wedin_samples(blocks, precomp_init_svd, init_signal_ranks, R=samples_per_computer)
    
    random_samples = np.concatenate([random_samples, comp_rand])
    
    for bn in blocks.keys():
        wedin_samples[bn] = np.concatenate([wedin_samples[bn], comp_wedin[bn]])
    

In [8]:
len(random_samples)
NUM_COMPS * samples_per_computer

1000

In [9]:
ajive= AJIVE(init_signal_ranks=init_signal_ranks,
              precomp_wedin_samples=wedin_samples,
              precomp_randdir_samples=random_samples)

%time ajive.fit(blocks=blocks, precomp_init_svd=precomp_init_svd)

CPU times: user 386 ms, sys: 82.1 ms, total: 468 ms
Wall time: 123 ms


AJIVE, joint rank: 1, block x indiv rank: 1, block y indiv rank: 2